In [ ]:
# for collab

import tensorflow as tf

def colab_connect():

    if tf.__version__!='2.13.0':
        %pip install tensorflow==2.13.0
        print('Stopping RUNTIME! Please run again.')
        import os
        os.kill(os.getpid(), 9)

    from google.colab import drive
    drive.mount('/content/drive/',force_remount=True)
    %cd /content/drive/My Drive/LOB/

    %pip install automodinit keras_tuner

try:
    import google.colab
    IS_COLAB = True
    colab_connect()
except:
    IS_COLAB =False

In [ ]:
if IS_COLAB:
    !nohup /usr/bin/python3 /content/drive/MyDrive/LOB/test_process.py &

In [ ]:
import numpy as np
from tqdm import tqdm
import keras

import data
from utilites import DataClass

seq_len = 100

In [ ]:
# Load data
row_data = (
    # data.load_dataset(horizon=4)
    data.load_saved_datas())

data.inspect_datas(row_data)

    Datas:
train     : x= (203799, 40)    | y= (203799,)      
val       : x= (50949, 40)     | y= (50949,)       
test      : x= (139586, 40)    | y= (139586,)      


In [ ]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

In [ ]:
from models import m_preln as test_model

In [ ]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=512,
    seq_len=seq_len,
)
(ds_train, ds_val, ds_test) =\
(datasets['train'], datasets['val'], datasets['test'])
data.inspect_datasets(datasets)

    Datasets:
train     : [398, 100, 40]
val       : [100, 100, 40]
test      : [273, 100, 40]


In [ ]:
# Build
model_name = 'base_model_low'
pars = DataClass(test_model.PARAMETRS)
model = test_model.build_model(**pars.Info_expanded)
model.summary()

In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.TensorBoard(
        f"Temp/callbacks/{model_name}",
        histogram_freq=1,
        update_freq=1,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f"Temp/callbacks/{model_name}/checkPoint",
        monitor="val_loss",
        verbose=0,
        save_best_only=False,
        save_weights_only=True,
        mode="auto",
        save_freq=50,
        options=None,
        initial_value_threshold=None,
    )
]

In [ ]:
# Train
model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
    callbacks=callbacks,
)